In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
%matplotlib inline
import re
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

import xgboost
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,SpatialDropout1D
from tensorflow.keras.layers import LSTM,Dropout
from keras.layers import Bidirectional
from tensorflow.keras.optimizers import RMSprop,Adam
from sklearn.model_selection import RandomizedSearchCV
from numpy import array
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test=pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train["target"].value_counts()

In [ ]:
sns.countplot("target",data=train)

# DATA PREPROCESSING

Let initiliaze our stemmer and tokenizer and do some data processing

In [ ]:
from textblob import TextBlob

In [ ]:
tweettoken = TweetTokenizer(strip_handles=True, reduce_len=True)

In [ ]:
lemmatizer=WordNetLemmatizer()

In [ ]:
stemmer=PorterStemmer()

In [ ]:
collect=[]
collecttest=[]
def preprocess(t,kpc):
    def form_sentence(tweet):
        tweet_blob = TextBlob(tweet)
        return ' '.join(tweet_blob.words)
    t=form_sentence(t)
    tee=re.sub('[^a-zA-Z]'," ",t)
    tee=tee.lower()
    res=tweettoken.tokenize(tee)
    for i in res:
        if i in stopwords.words('english'):
            res.remove(i)
    rest=[]
    for k in res:
        rest.append(stemmer.stem(k))
    ret=" ".join(rest)
    if kpc==1:
        collect.append(ret)
    elif kpc==0:
        collecttest.append(ret)

In [ ]:
def splitpro(t,q,m):
         for j in range(q):
                 preprocess(t["text"].iloc[j],m)

In [ ]:
len(train)

In [ ]:
len(test)

In [ ]:
splitpro(train,7613,1)

In [ ]:
splitpro(test,3263,0)

In [ ]:
len(collect)

In [ ]:
len(collecttest)

In [ ]:
collect[:5]

In [ ]:
collecttest[:5]

In [ ]:
val=train["target"].values

In [ ]:
val

In [ ]:
oneh=[]
oneht=[]
def hot(cc,k):
    for i in cc:
        if k==1:
            oneh.append(one_hot(i,500))
        elif k==0:
            oneht.append(one_hot(i,500))

In [ ]:
hot(collect,1)

In [ ]:
hot(collecttest,0)

In [ ]:
len(oneh[0])

In [ ]:
max=0
for i in oneh:
    tq=len(i)
    if tq>max:
        max=tq
print(max)

In [ ]:
sent=38
emoneh=pad_sequences(oneh,padding="post",maxlen=sent)
emoneht=pad_sequences(oneht,padding="post",maxlen=sent)

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(emoneh,val,train_size=0.80,random_state=42)

In [ ]:
model=Sequential()
model.add(Embedding(500,15,input_length=sent))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(100))
model.add(Dropout(0.1))
model.add(Dense(1, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(xtrain,ytrain,validation_data=(xtest,ytest),epochs=50,batch_size=300,verbose=2)

In [ ]:
tttt=model.predict(emoneht)

In [ ]:
tttt

In [ ]:
ttttt=tttt.flatten()

In [ ]:
r = pd.Series(ttttt,name="target")

In [ ]:
sample=pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

In [ ]:
sample.head()

In [ ]:
t=test["id"]

In [ ]:
t

In [ ]:
submiss = pd.concat([pd.Series(t,name = "id"),r],axis = 1)

In [ ]:
submiss

In [ ]:
submiss.to_csv("disasml20.csv",index=False)